# Video Capture and Processing

In [2]:
#Importing the libraries
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

## Video Capture Using OpenCV

In [3]:
#video capture using opencv
cap = cv.VideoCapture(0)

#display the frame
while True: 
    #Orignal frame
    ret, frame = cap.read()
    cv.imshow('frame', frame)
    
    #Convert to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cv.imshow('gray', gray)

    #Add a delay of 10ms and press q to quit
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
#Release the capture
cap.release()
cv.destroyAllWindows()

## Mask for a Video Capture

In [4]:
# Mask for a video capture
# Detecting yellow color in the video using HSV color space
cap = cv.VideoCapture(0)

while True:

    #Orignal frame
    #ret - return value, frame - frame
    ret, frame = cap.read()
    cv.imshow('frame', frame)
    
    #Convert to hsv
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    cv.imshow('hsv', hsv)

    #lower and upper bound for the mask
    lower_yellow = np.array([10,100,50])
    upper_yellow = np.array([70,255,255])

    #mask
    mask = cv.inRange(hsv, lower_yellow, upper_yellow)
    cv.imshow('mask', mask)

    #bitwise and
    res = cv.bitwise_and(frame, frame, mask = mask)
    cv.imshow('res', res)

    #Add a delay of 10ms and press q to quit
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

#Release the capture
cap.release()   
cv.destroyAllWindows()

## For an External Video Camera

In [5]:
#Importing the libraries
import cv2 as cv

#video capture using opencv
cap = cv.VideoCapture(1)

#display the frame
while True: 
    #Orignal frame
    ret, frame = cap.read()
    cv.imshow('frame', frame)
    
    #Convert to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cv.imshow('gray', gray)

    #Add a delay of 10ms and press q to quit
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
#Release the capture
cap.release()
cv.destroyAllWindows()

## 2 Camera Feeds at Once

In [24]:
import cv2 as cv
import numpy as np

capture_0 = cv.VideoCapture(0)
capture_1 = cv.VideoCapture(1)


while True:

    #Orignal frame
    #ret - return value, frame - frame
    _, frame_1 = capture_1.read()
    frame_1 = cv.GaussianBlur(frame_1, (5,5), 0)
    _, frame_0 = capture_0.read()
    
    # resize the frame_0
    frame_0 = cv.resize(frame_0, (640,480))
    cv.imshow('frame_0', frame_0)

    # resize the frame_1
    frame_1 = cv.resize(frame_1, (640,480))
    cv.imshow('frame_1', frame_1)
    
    # Convert frame_0 to hsv
    hsv = cv.cvtColor(frame_0, cv.COLOR_BGR2HSV)
    cv.imshow('hsv_0', hsv)

    # Convert frame_1 to hsv
    hsv = cv.cvtColor(frame_1, cv.COLOR_BGR2HSV)
    cv.imshow('hsv_1', hsv)

    #lower and upper bound for the mask
    lower_yellow = np.array([10,100,50])
    upper_yellow = np.array([70,255,255])

    #mask
    #mask = cv.inRange(hsv, lower_yellow, upper_yellow)
    #cv.imshow('mask', mask)

    #bitwise and
    #res = cv.bitwise_and(frame_1, frame_1, mask = mask)
    #cv.imshow('res', res)

    #Add a delay of 10ms and press q to quit
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

#Release the capture
cap.release()   
cv.destroyAllWindows()

In [ ]:
#!/usr/bin/env python

import numpy as np
import cv2

def read_rgb_image(image_name, show):
    rgb_image = cv2.imread(image_name)
    if show: 
        cv2.imshow("RGB Image",rgb_image)
    return rgb_image

def filter_color(rgb_image, lower_bound_color, upper_bound_color):
    #convert the image into the HSV color space
    hsv_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2HSV)
    cv2.imshow("hsv image",hsv_image)

    #find the upper and lower bounds of the yellow color (tennis ball)
    yellowLower =(30, 150, 100)
    yellowUpper = (50, 255, 255)

    #define a mask using the lower and upper bounds of the yellow color 
    mask = cv2.inRange(hsv_image, lower_bound_color, upper_bound_color)

    return mask

def getContours(binary_image):      
    #_, contours, hierarchy = cv2.findContours(binary_image, 
    #                                          cv2.RETR_TREE, 
    #                                           cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv2.findContours(binary_image.copy(), 
                                            cv2.RETR_EXTERNAL,
	                                        cv2.CHAIN_APPROX_SIMPLE)
    return contours


def draw_ball_contour(binary_image, rgb_image, contours):
    black_image = np.zeros([binary_image.shape[0], binary_image.shape[1],3],'uint8')
    
    for c in contours:
        area = cv2.contourArea(c)
        perimeter= cv2.arcLength(c, True)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        if (area>100):
            cv2.drawContours(rgb_image, [c], -1, (150,250,150), 1)
            cv2.drawContours(black_image, [c], -1, (150,250,150), 1)
            cx, cy = get_contour_center(c)
            cv2.circle(rgb_image, (cx,cy),(int)(radius),(0,0,255),1)
            cv2.circle(black_image, (cx,cy),(int)(radius),(0,0,255),1)
            cv2.circle(black_image, (cx,cy),5,(150,150,255),-1)
            print ("Area: {}, Perimeter: {}".format(area, perimeter))
    print ("number of contours: {}".format(len(contours)))
    cv2.imshow("RGB Image Contours",rgb_image)
    cv2.imshow("Black Image Contours",black_image)

def get_contour_center(contour):
    M = cv2.moments(contour)
    cx=-1
    cy=-1
    if (M['m00']!=0):
        cx= int(M['m10']/M['m00'])
        cy= int(M['m01']/M['m00'])
    return cx, cy

def main():
    image_name = "C:\Python\ipmv\Scripts\Image_Processing_NoteBooks\Images\Tennis_Racket_and_Balls.jpg"
    yellowLower =(30, 150, 100)
    yellowUpper = (50, 255, 255)
    rgb_image = read_rgb_image(image_name, True)
    binary_image_mask = filter_color(rgb_image, yellowLower, yellowUpper)
    contours = getContours(binary_image_mask)
    draw_ball_contour(binary_image_mask, rgb_image,contours)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

main()

cv2.waitKey(0)
cv2.destroyAllWindows()


## Thumbs Up Detection

Still working on

In [5]:
# Thumbsup detectioin using a mask
import cv2 
import numpy as np

# Function to detect the red ball
def detect_red_ball(frame):

    frame = cv2.GaussianBlur(frame, (5, 5), 0)

    # Convert the frame from BGR to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper HSV thresholds for red color
    # Note: Hue range is [0, 179] and Saturation range is [0, 255] Value range is [0, 255]
    # Need to adjust these values to detect the exact red colour of the ball
    lower_red = np.array([0, 50, 20])
    upper_red = np.array([30, 255, 200])

    # Create a mask to extract the red regions
    mask1 = cv2.inRange(hsv_frame, lower_red, upper_red)

    # Define the lower and upper HSV thresholds for red color (shades of red)
    lower_red = np.array([160, 100, 100])
    upper_red = np.array([179, 255, 255])

    # Create another mask to extract the red regions (shades of red)
    mask2 = cv2.inRange(hsv_frame, lower_red, upper_red)

    # Combine the two masks to get the final mask for red regions
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Apply the mask to the original frame
    red_ball = cv2.bitwise_and(frame, frame, mask=red_mask)
    #red_ball = cv2.bitwise_and(frame, frame, mask=mask1)

    # draw a circle around all the red balls in the frame
    contours, hierarchy = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #contours, hierarchy = cv2.findContours(mask1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    area_array=[]
    for contour in contours:
        #area = cv2.contourArea(contour)
        perimeter= cv2.arcLength(contour, True)

        if 400 < perimeter < 800: #can adjust this value with the area of the red ball
        #if 100< area < 200: #can adjust this value with the area of the red ball
            #area_array.append(area)
            area_array.append(perimeter)    
            x, y, w, h = cv2.boundingRect(contour)
            cv2.circle(red_ball, (int(x + w/2), int(y + h/2)), int((w + h)/4), (0, 255, 0), 2)
    
    # display the number of red balls in the frame with area greater than 500
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(red_ball, 'Number of red balls: ' + str(len(area_array)), (10, 50), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

    return red_ball


# Open a video capture object or use the camera
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the video source
    ret, frame = cap.read()
    
    if not ret:
        break

    # Detect the red ball in the frame
    red_ball = detect_red_ball(frame)

    # Display the original frame and the red ball detection
    cv2.imshow('Original Frame', frame)
    cv2.imshow('Red Ball Detection', red_ball)

    # Exit when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()



## Using Mediapipe

In [13]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils

# Open a video capture
cap = cv2.VideoCapture(0)  # Use the default camera (change the parameter if using an external camera)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        continue

    # Flip the frame horizontally for a later selfie-view display
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect gestures
    results = holistic.process(frame_rgb)

    if results.pose_landmarks and results.left_hand_landmarks:
        left_hand_landmarks = results.left_hand_landmarks.landmark

        # Assuming that the thumb and index finger landmarks are detected correctly
        thumb_tip = left_hand_landmarks[4]
        index_tip = left_hand_landmarks[8]

        # Calculate the Euclidean distance between thumb and index finger tips
        distance = ((thumb_tip.x - index_tip.x) ** 2 + (thumb_tip.y - index_tip.y) ** 2) ** 0.5

        # Adjust this threshold to define when a thumbs-up is detected
        if distance < 0.05:
            cv2.putText(frame, "Thumbs Up", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Draw hand landmarks on the frame
    mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    # Display the frame
    cv2.imshow('Thumbs-Up Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2 as cv
import numpy as np

capture_0 = cv.VideoCapture(0)
capture_1 = cv.VideoCapture(1)


while True:

    #Orignal frame
    #ret - return value, frame - frame
    _, frame_1 = capture_1.read()
    frame_1 = cv.GaussianBlur(frame_1, (5,5), 0)
    _, frame_0 = capture_0.read()
    
    # resize the frame_0
    frame_0 = cv.resize(frame_0, (640,480))
    cv.imshow('frame_0', frame_0)

    # resize the frame_1
    frame_1 = cv.resize(frame_1, (640,480))
    cv.imshow('frame_1', frame_1)
    
    # Convert frame_0 to hsv
    hsv = cv.cvtColor(frame_0, cv.COLOR_BGR2HSV)
    cv.imshow('hsv_0', hsv)

    # Convert frame_1 to hsv
    hsv = cv.cvtColor(frame_1, cv.COLOR_BGR2HSV)
    cv.imshow('hsv_1', hsv)

    #lower and upper bound for the mask
    lower_yellow = np.array([10,100,50])
    upper_yellow = np.array([70,255,255])

    #mask
    #mask = cv.inRange(hsv, lower_yellow, upper_yellow)
    #cv.imshow('mask', mask)

    #bitwise and
    #res = cv.bitwise_and(frame_1, frame_1, mask = mask)
    #cv.imshow('res', res)

    #Add a delay of 10ms and press q to quit
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

#Release the capture
cap.release()   
cv.destroyAllWindows()

## Depth of the Camera

Tried with my webcamera and mobile phone camera. They don't give a correct result at all.

In [22]:
import cv2
import numpy as np

# Initialize the video capture with the appropriate source (e.g., 0 for the default camera).
cap_i = cv2.VideoCapture(0)  # IRIUN camera.
cap_f = cv2.VideoCapture(1)  # Default camera.

while True:
    ret_i, depth_frame_i = cap_i.read()  # Read a depth frame.
    ret_f, depth_frame_f = cap_f.read()  # Read a depth frame.

    depth_frame_i_gray = cv2.cvtColor(depth_frame_i, cv2.COLOR_BGR2GRAY)
    depth_frame_f_gray = cv2.cvtColor(depth_frame_f, cv2.COLOR_BGR2GRAY)

    stereo = cv2.StereoBM.create(numDisparities=16, blockSize=15)
    disparity = stereo.compute(depth_frame_f_gray,depth_frame_i_gray)

    # Display the depth frame and its colormap.
    cv2.imshow('IRIUN Camera', depth_frame_i)
    cv2.imshow('Front Web Camera', depth_frame_f)
    cv2.imshow('Disparity', disparity)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
